In [ ]:
import sys

import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def imshow(img, ax=None, vmin=None, vmax=None, figsize=(7, 7), external=False, title=""):
    if external:
        cv.imshow("imshow", img)
        
        while 1:
            if cv.waitKey(0) & 0xFF == ord("q"):
                break
        
        cv.destroyAllWindows()
        return
    
    gray = False

    if len(img.shape) == 2:
        gray = True
    if img.shape == 3 and img.shape[-1] == 1:
        gray = True

    # trying to remove as much as possible
    if ax is None:
        _, ax = plt.subplots(figsize=figsize)
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    ax.set_title(title)
    ax.autoscale(tight=True)
    
    if gray:
        ax.imshow(img, cmap="gray", vmin=vmin, vmax=vmax)
    else:
        ax.imshow(img[:, :, ::-1], vmin=vmin, vmax=vmax)

## Color Correction

### Contrast Limited Adaptive Histogram Equalization (CLAHE)

In [ ]:
def clahe(img, grid_size=7):
    clh_obj = cv.createCLAHE(clipLimit=2.0, tileGridSize=(grid_size, grid_size))
    lab_space = cv.cvtColor(img, cv.COLOR_BGR2LAB)
    img_channels = cv.split(lab_space)
    img_channels[0] = clh_obj.apply(img_channels[0])
    return cv.cvtColor(cv.merge(img_channels), cv.COLOR_LAB2BGR)

In [ ]:
img = cv.imread("images/fullbody.jpg")
clahe_img = clahe(img)
imshow(np.hstack([img, clahe_img]), figsize=(15, 15))

## Face and Body Detection

In [ ]:
def draw_box(img, bounding_box):
    img = img.copy()
    x, y, w, h = bounding_box
    
    return cv.rectangle(img, (x, y), (x+w, y+h), (255, 0, 0), 3)

In [ ]:
!wget https://github.com/opencv/opencv/raw/master/data/haarcascades/haarcascade_frontalface_default.xml -O haarcascade_frontalface_default.xml

In [ ]:
def get_face(img):
    img = img.copy()
    img_gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

    face_cascade = cv.CascadeClassifier()

    if not face_cascade.load(cv.samples.findFile("./haarcascade_frontalface_default.xml")):
        print("--(!)Error loading face cascade")
        sys.exit(1)

    try:
        face = face_cascade.detectMultiScale(img_gray)[0]
    except IndexError as e:
        print(e)
        print("--(!)No faces detected")
        sys.exit(1)
    
    return face

In [ ]:
def get_body(img, face_bounds):
    # image height
    hi = img.shape[0]

    # face bounds
    xf, yf, wf, hf = face_bounds
    
    # x_body_left = x_face_left - wf
    x = xf - wf
    # y_body_top = y_face_top - hf
    y = yf - hf
    w = 3 * wf
    h = hi - y
    
    return (x, y, w, h)

In [ ]:
face_bounds = get_face(clahe_img)
body_bounds = get_body(clahe_img, face_bounds)
boxed_face = draw_box(clahe_img, face_bounds)
boxed_body = draw_box(boxed_face, body_bounds)
imshow(boxed_body)